# Set up

In [1]:
import numpy as np
import pandas as pd
from typing import List, Tuple

In [2]:
RNG = np.random.default_rng(1030)

LOW, HIGH = 0.1, 5.0
BASELINE = 2.5
CENTRES = [1.0, 1.1, 1.5, 2.0, 3.0]
REPLICATES_PER_REGION = 10

In [3]:
def sample_trunc_bvn(mean: np.ndarray, cov: np.ndarray, low: float, high: float) -> np.ndarray:
    while True:
        x = RNG.multivariate_normal(mean, cov)
        if low <= x[0] <= high and low <= x[1] <= high:
            return x

def build_datasets_for_centre(
    m: float,
    sd2: float = 0.1,
    rho: float = 0.3,
    baseline_jitter: float = 1e-2,
    jitter_mode: str = "random"
) -> pd.DataFrame:
    sd = np.sqrt(sd2)
    cov = np.array([[sd**2, rho*sd**2],
                    [rho*sd**2, sd**2]])
    rows = []
    for region in range(1, 6):
        for _ in range(REPLICATES_PER_REGION):
            mean = np.array([m, m])
            a_r, b_r = sample_trunc_bvn(mean, cov, LOW, HIGH)

            params = []
            for r in range(1, 6):
                if r == region:
                    # 当前被“激活”的区域：用真正采样到的那一对
                    params.extend([a_r, b_r])
                else:
                    # 其他区域：本来是 BASELINE，这里加一个很小的扰动
                    if jitter_mode == "random":
                        # 随机小抖动，零均值，幅度由 baseline_jitter 控制
                        eps_a = np.random.uniform(-baseline_jitter, baseline_jitter)
                        eps_b = np.random.uniform(-baseline_jitter, baseline_jitter)
                        params.extend([BASELINE + eps_a, BASELINE + eps_b])
                    else:
                        # 固定的小偏移，保证不等于 BASELINE
                        params.extend([BASELINE + baseline_jitter,
                                       BASELINE + baseline_jitter])

            rows.append(params)

    df = pd.DataFrame(
        rows,
        columns=[f"{ab}{r}" for r in range(1, 6) for ab in ("a", "b")]
    )
    return df



In [4]:

df = build_datasets_for_centre(1.2, sd2=0.05, rho=0.3, baseline_jitter = 0)

In [5]:
df.shape

(50, 10)

In [6]:
df

,a1,b1,a2,b2,a3,b3,a4,b4,a5,b5
0,1.503931,1.633299,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
1,1.070694,0.829028,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
2,0.672183,0.882623,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
3,0.957749,1.107041,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
4,1.180926,1.138803,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
5,1.415496,1.137106,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
6,1.380614,1.301490,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
7,1.447955,1.075303,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
8,0.954817,0.817563,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
9,0.739581,0.818898,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000


In [11]:
df = pd.DataFrame({f'col_{i}': [5] for i in range(10)})

In [12]:
df

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,5,5,5,5,5,5,5,5,5,5


In [13]:
df = df.to_numpy()
m, n = df.shape


new_data = np.zeros((m, 34))


mapping = {
    0: [4, 16],
    1: [5, 17],
    2: [10, 22],
    3: [11, 23],
    4: [8, 6, 20, 18],
    5: [9, 7, 21, 19],
    6: [2, 0, 14, 12],
    7: [3, 1, 15, 13],
    8: [24, 26, 28, 30, 32],
    9: [25, 27, 29, 31, 33]
}

for orig_col, new_cols in mapping.items():
    for new_col in new_cols:
        new_data[:, new_col] = df[:, orig_col]

In [14]:
# np.savetxt("LocalDisease/X_3.txt", new_data, fmt='%0.4f')
np.savetxt("X_5_max.txt", new_data, fmt='%0.4f')

In [ ]:
X_train = pd.read_csv('X_1_1.txt', header=None, delimiter=' ').values

m = X_train.shape[0]

mapping = {
    0: [4, 16],
    1: [5, 17],
    2: [10, 22],
    3: [11, 23],
    4: [8, 6, 20, 18],
    5: [9, 7, 21, 19],
    6: [2, 0, 14, 12],
    7: [3, 1, 15, 13],
    8: [24, 26, 28, 30, 32],
    9: [25, 27, 29, 31, 33]
}

X_all_recovered = np.zeros((m, len(mapping)))


for orig_col, new_cols in mapping.items():

    X_all_recovered[:, orig_col] = X_train[:, new_cols].mean(axis=1)

X_train = np.around(X_all_recovered, decimals=4)

In [ ]:
np.savetxt("X_1_1.csv", X_train, delimiter=",", fmt="%.4f")

# Check

In [2]:
Y_train_pca = pd.read_csv('Y_train_std_pca.csv', header=None, delimiter=',').values
Y_test_pca = pd.read_csv('Y_test_std_pca.csv', header=None, delimiter=',').values

Y_data_1_1_pca = pd.read_csv('Y_data_1_1_pca.csv', header=None, delimiter=',').values
Y_data_1_5_pca = pd.read_csv('Y_data_1_5_pca.csv', header=None, delimiter=',').values

Y_edge_std_pca = pd.read_csv('Y_edge_std_pca.csv', header=None, delimiter=',').values

Y_data_1_2_jitter_B_2_5_pca = pd.read_csv('Y_data_1_2_jitter_B_2_5_pca.csv', header=None, delimiter=',').values

In [4]:
col_min = np.nanmin(Y_edge_std_pca, axis=0)
col_max = np.nanmax(Y_edge_std_pca, axis=0)


((Y_data_1_2_jitter_B_2_5_pca >= col_min) & (Y_data_1_2_jitter_B_2_5_pca <= col_max)).sum(axis=1)

array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 16, 17, 17, 17, 17, 17, 17, 16, 17, 16, 17, 18, 18, 18,
       18, 17, 18, 18, 17, 18, 18, 17, 17, 17, 18, 17, 18, 17, 18, 17])

In [ ]:
X_train = pd.read_csv('../Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('../Data/X_test.csv', header=None, delimiter=',').values
X_edge = pd.read_csv('../Data/X_edge.csv', header=None, delimiter=',').values

In [ ]:
X_all = np.vstack([X_train, X_edge])